In [1]:
import ee
ee.Initialize()

In [2]:
from iteru import *
from IPython.display import display
import ipywidgets
import datetime

In [3]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [4]:
aoi = Map.aoi

In [5]:
demvis = {'min':488,'max':1863,'palette':['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
}

In [6]:
dataset = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2')\
           .filter(ee.Filter.bounds(aoi))\
           .select('DSM')\
           .median()\
           .clip(aoi)

In [7]:
#Map.add_layer_widgets(dataset,demvis,'GRED_DEM')

In [8]:
S2 = ee.ImageCollection("COPERNICUS/S2_SR")\
     .filter(ee.Filter.bounds(aoi))\
     .filter(ee.Filter.date('2022-01-01','2022-02-01'))\
     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5))

In [9]:
S2.size().getInfo()

10

In [10]:
vis_param = get_vis_params(S2)

In [11]:
S2_median = S2.median().clip(aoi)

In [12]:
Map.add_layer_widgets(S2_median,vis_param,'S2_Median')

In [13]:
ndwi = S2_median.normalizedDifference(['B3', 'B8'])

In [14]:
ndwiMasked = ndwi.updateMask(ndwi.gte(0))

In [15]:
#Map.add_layer_widgets(ndwiMasked,{'min':0,'max':0.5,'palette': ['00FFFF', '0000FF']},'NDWI')

In [16]:
v = ee.Image(1).updateMask(ndwiMasked).reduceToVectors(
           geometry = ndwiMasked.geometry(),
           crs =  ndwiMasked.projection(),
           scale = 10 , 
           geometryType = 'polygon',
           eightConnected = False,
           labelProperty =  'water_cover',
           bestEffort = True
)

In [17]:
def calc_area(feature):
    area = feature.geometry().area(maxError = 1)
    
    return feature.set({'Area':area})

In [18]:
v = v.map(calc_area)

In [19]:
v = ee.Feature(v.sort('Area', False).first())

In [20]:
Map.add_layer_widgets(v,{'color':'red'},'Vector')

In [21]:
lake_dem = dataset.clip(v)

In [22]:
x = lake_dem.updateMask(lake_dem.gte(170))

In [23]:
Map.add_layer_widgets(x,demvis,'X')

In [24]:
ele = lake_dem.reduceRegion(
                         reducer = ee.Reducer.max(),
                         geometry = lake_dem.geometry(), 
                         scale = 10, 
                        maxPixels = 1e10)

In [25]:
water_level = ele.getInfo()['DSM']

In [26]:
water_level

604

In [27]:
import numpy as np

In [28]:
#latlng = ee.Image.pixelLonLat().addBands(lake_dem)
latlng = lake_dem.reduceRegion(reducer=ee.Reducer.toList(), geometry=v.geometry(), maxPixels=1e11, scale=30)

In [29]:
elev_values = np.array((ee.Array(latlng.get("DSM")).getInfo()),dtype=np.int64)

In [30]:
elev_values.size

430310

In [40]:
volume = water_level - elev_values[elev_values < water_level]

In [41]:
(volume.sum()*900)/1e9

18.5347368

In [33]:
url = ee.FeatureCollection(v).getDownloadURL('geojson')

In [34]:
print(url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/80429ca947e91c7b30c0c28586d4eb69-741312043cd88ebe476a0efdd58b4dae:getFeatures


In [36]:
url2 = lake_dem.getDownloadURL({'name': 'MyPieceOfDEM', 'crs': 'EPSG:32636', 'scale': 35})

In [37]:
print(url2)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/b7a3dda2af71fc2db9aaebd76721b46f-f610c733db9acda7a2ff862321ca02c9:getPixels


In [42]:
SAR = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filterBounds(aoi)\
    .filterDate('2022-01-01','2022-02-01')\
    .select(['VV','VH'])

In [47]:
sar = SAR.median().clip(aoi)

In [48]:
Map.add_layer_widgets(sar.select(['VV']),{'min': -20, 'max': -7},'SAR_TEST_1')